In [9]:

import os
from os.path import dirname as up
from typing import Any, Dict, Type, cast
import pytorch_lightning as pl
from omegaconf import OmegaConf
import shutil
import time

from MarshModel import MarshModel
import smp_metrics

import pandas as pd
from sharp_dataloader import GenMARSH, RandomRotation, Resize
import torch
from torch.utils.data import DataLoader
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger
import imgaug as ia
import imgaug.augmenters as iaa
import torchvision.transforms as transforms
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import itertools
import numpy as np
from sharp_trainer import SemanticSegmentationTask

In [10]:
import sys
path_cur = os.path.abspath(os.getcwd())
root_path = up(path_cur)
sys.path.append(root_path)
from utils import metrics

In [11]:

torch.manual_seed(0)
    
conf = OmegaConf.load("./config_smp.yaml")
conf_dict = OmegaConf.to_object(conf.experiment)
conf_dict = cast(Dict[Any, Dict[Any, Any]], conf_dict)

# prepare data for training
dl_kwargs = conf_dict["dataloader"]
tmi_file = dl_kwargs["labelpath"]
data_root = dl_kwargs["data_root_dir"]

transform_train = transforms.Compose([transforms.ToTensor(),
                                Resize(256)])

# load data
dataset = GenMARSH(tmi_file, data_root, transform=transform_train, normalization=dl_kwargs['normalization'], ndvi=dl_kwargs['ndvi'], ndwi=dl_kwargs['ndwi'], datasource=dl_kwargs['datasource'])

train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(0))

trainloader = DataLoader(train_dataset, 
                    batch_size = dl_kwargs["batch_size"], 
                    shuffle = True,
                    num_workers = dl_kwargs["num_workers"],
                    pin_memory = False)

testloader = DataLoader(test_dataset, 
                    batch_size = dl_kwargs["batch_size"], 
                    shuffle = False,
                    num_workers = dl_kwargs["num_workers"],
                    pin_memory = False)
    


TypeError: __init__() got multiple values for argument 'normalization'

In [4]:
len(dataset)

3608

In [5]:
# get some random training images
dataiter = iter(trainloader)
batch = dataiter.next()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def show_image(image, label):

    rgb_img = np.stack((image[2,:,:], image[1,:,:], image[0,:,:]), axis=2).astype(np.uint8)
    gray_img = image[0,:,:][:, :,np.newaxis]
    
    label = np.transpose(label, (1,2,0))
    
    f = plt.figure()
    f.add_subplot(1,2,1)
    plt.imshow(rgb_img)
    
    f.add_subplot(1,2,2)
    plt.imshow(gray_img)

    plt.show(block=True)
    

def display_NAIP(image, label):
    
    # reshape data
    ndata = np.moveaxis(image.numpy(), 0, -1)  # ndata is of shape (40, 40, 13)
    # select RGB (or any other combination)
    rgb = ndata[...,[2,1,0]]
    
#     nlabel = np.moveaxis(label.numpy(), 0, -1)
    nlabel = label.numpy()
    
    f = plt.figure()
    f.add_subplot(1,2,1)
    plt.imshow(rgb)
    
    f.add_subplot(1,2,2)
    plt.imshow(nlabel)
    plt.show(block=True)


for i in range(50):
    
    sample = dataset[i]
    display_NAIP(**sample)

    if i == 50:
        break

In [ ]:
# # load model parameters
# model_kwargs = conf_dict["module"]
# model = MarshModel(**model_kwargs)


# experiment_name = f"{model_kwargs['arch']}_{model_kwargs['encoder_name']}_0.001"
# experiment_dir = os.path.join(dl_kwargs["out_dir"], experiment_name)
# logger = TensorBoardLogger(experiment_dir, name="models")

# early_stop_callback = EarlyStopping(monitor="valid_dataset_iou", min_delta=0.00, patience=5, verbose=False, mode="max")
# checkpoint_callback = ModelCheckpoint(
#         monitor='valid_dataset_iou', dirpath=experiment_dir, save_top_k=1, save_last=True, mode='max')

# trainer = pl.Trainer(
#         callbacks=[early_stop_callback, checkpoint_callback],
#         logger=logger,
#         default_root_dir=experiment_dir,
#         min_epochs=1,
#         max_epochs=100,
#         accelerator="gpu",
#         devices=[3])

# trainer.fit(
#     model, 
#     train_dataloaders=trainloader, 
#     val_dataloaders=testloader,
# )

In [6]:
def copy_dataloader_setting(working_dir, filepath):

    versions = [i for i in os.listdir(os.path.join(working_dir, 'models'))]
    curr_version = versions[-1]
    
    # destination folder
    dest = os.path.join(working_dir, 'models', curr_version)
    shutil.copy2(filepath, dest)


In [7]:
monitor_options = ['val_loss']
model_options = ['unet', 'fpn', 'pspnet', 'deeplabv3', 'deeplabv3plus', 'pan', 'manet', 'linknet']
encoder_options = ['resnet34']
lr_options = [1e-4]
loss_options = ["ce"]
weight_init_options = ["imagenet"]
in_channel = 5
out_channel = 2


for (model, encoder, lr, loss, weight_init, monitor_state) in itertools.product(
        model_options,
        encoder_options,
        lr_options,
        loss_options,
        weight_init_options,
        monitor_options):
    
    experiment_name = f"{monitor_state}_{model}_{encoder}_{lr}_{loss}_{weight_init}"
    
    print(experiment_name)
    
    experiment_dir = os.path.join(dl_kwargs["out_dir"], experiment_name)
    logger = TensorBoardLogger(experiment_dir, name="models")

    
    checkpoint_callback = ModelCheckpoint(
        monitor=monitor_state, dirpath=experiment_dir, save_top_k=1, save_last=True, mode='min')

    early_stopping_callback = EarlyStopping(monitor=monitor_state, min_delta=0.00, patience=5, mode='min')
    
    model = SemanticSegmentationTask(
                    segmentation_model=model,
                    encoder_name=encoder,
                    encoder_weights=weight_init,
                    learning_rate=lr,
                    in_channels=in_channel,
                    num_classes=out_channel,
                    learning_rate_schedule_patience=6,
                    ignore_index=0,
                    loss=loss,
                    imagenet_pretraining=True)

    trainer = pl.Trainer(
                callbacks=[checkpoint_callback, early_stopping_callback],
                logger=logger,
                default_root_dir=experiment_dir,
                min_epochs=1,
                max_epochs=100,
                accelerator="gpu",
                devices=[3])

    trainer.fit(model, trainloader, testloader)

    
    

val_loss_unet_resnet34_0.0001_ce_imagenet


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ../outputs/outputs_binary_tmi_naip/val_loss_unet_resnet34_0.0001_ce_imagenet/models
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | Unet             | 24.4 M
1 | loss          | CrossEntropyLoss | 0     
2 | train_metrics | MetricCollection | 0     
3 | val_metrics   | MetricCollection | 0     
4 | test_metrics  | MetricCollection | 0     
---------------------------------------------------
24.4 M    Trainable params
0         Non-trainable params
24.4 M    Total params
97.771    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/envs/vims/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of size: : [32, 1, 256, 256]

In [ ]:
# model_options = ['unet', 'fpn', 'pspnet', 'deeplabv3', 'deeplabv3plus', 'pan', 'manet', 'linknet']
# encoder_options = ['resnet34']
# lr_options = [1e-4]
# weight_init_options = ["imagenet"]
# in_channel = 5
# out_channel = 1

# for (model, encoder, lr) in itertools.product(
#         model_options,
#         encoder_options,
#         lr_options):
    
#     experiment_name = f"MarshModel_{model}_{encoder}_{lr}"
    
#     print(experiment_name)
    
#     experiment_dir = os.path.join(dl_kwargs["out_dir"], experiment_name)
#     logger = TensorBoardLogger(experiment_dir, name="models")

#     early_stop_callback = EarlyStopping(monitor="valid_dataset_iou", min_delta=0.00, patience=5, verbose=False, mode="max")
#     checkpoint_callback = ModelCheckpoint(
#             monitor='valid_dataset_iou', dirpath=experiment_dir, save_top_k=1, save_last=True, mode='max')
    
#     # load model parameters
#     model = MarshModel(model, encoder_name=encoder, in_channels=in_channel, out_classes=out_channel)

#     trainer = pl.Trainer(
#         callbacks=[early_stop_callback, checkpoint_callback],
#         logger=logger,
#         default_root_dir=experiment_dir,
#         min_epochs=1,
#         max_epochs=100,
#         accelerator="gpu",
#         devices=[3])

#     trainer.fit(
#         model, 
#         train_dataloaders=trainloader, 
#         val_dataloaders=testloader,
#     )
    

#     config_path = os.path.join(path_cur, 'config_smp.yaml')
#     copy_dataloader_setting(experiment_dir, config_path)
    

In [ ]:
os.path.join(root_path, 'outputs', 'outputs_binary_tmi_naip')

In [ ]:
for m in os.listdir(os.path.join(root_path, 'outputs', 'outputs_binary_tmi_naip')):
    if not m.endswith('ipynb_checkpoints'):
        print("Working on {}".format(m))
        start_time = time.time()
        mname = m.split('_')[1]        
        mpath = os.path.join(root_path, 'outputs', 'outputs_binary_tmi_naip', m, 'last.ckpt')
        model = MarshModel.load_from_checkpoint(mpath)
        model.eval()
        

        y_true = []
        y_predicted = []
        
        with torch.no_grad():

            for batch in testloader:

                image = batch["image"]
                target = batch["label"]

                logits = model(image) # predicted
                print(logits[0])
                print(target[0])
#                 logits = torch.nn.functional.softmax(logits, dim=1).argmax(1)

#                 logits = logits.reshape(-1)
#                 target = target.reshape(-1)

                # Accuracy metrics only on annotated pixels (Check on the dimension changes)

#                 mask = target != 0 # 0 is the background
#                 logits = logits[mask]
#                 target = target[mask]

#                 y_predicted += logits.tolist()
#                 y_true += target.tolist()
        
#         y_predicted = np.array(y_predicted)
#         y_true = np.array(y_true)
        
#         print(len(y_predicted[y_predicted==1]))
#         print(len(y_predicted[y_predicted==0]))
#         print(len(y_true[y_true==1]))
#         print(len(y_true[y_true==0]))
        
        
        
#         acc = metrics.Evaluation(y_predicted, y_true)
#         print("Evaluation: " + str(acc))
#         """
#         Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label 
#         You being i-th class and predicted label being j-th class.
#         """
#         try:
#             test_labels = ['no_marsh', 'marsh']
#             conf_mat = metrics.confusion_matrix(y_true, y_predicted, test_labels)
#             print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
#         except:
#             labels = ['Background', 'no_marsh', 'marsh']
#             conf_mat = metrics.confusion_matrix(y_true, y_predicted, labels)
#             print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
            

#         end_time = time.time()
#         time_diff = end_time - start_time
#         print("time spent: {}".format(time_diff))




In [ ]:


for m in os.listdir(os.path.join(root_path, 'outputs')):
    if not m.endswith('ipynb_checkpoints'):
        print("Working on {}".format(m))
        start_time = time.time()
        mpath = os.path.join(root_path, 'outputs', m, 'last.ckpt')
        model_load = SemanticSegmentationTask.load_from_checkpoint(mpath)
        model_load.eval()
        
        y_true = []
        y_predicted = []
        
        with torch.no_grad():

            for batch in testloader:

                image = batch["image"]
                target = batch["label"]

                logits = model_load(image) # predicted
                logits = torch.nn.functional.softmax(logits, dim=1).argmax(1)

                logits = logits.reshape(-1)
                target = target.reshape(-1)

                # Accuracy metrics only on annotated pixels (Check on the dimension changes)

                mask = target != 0 # 0 is the background
                logits = logits[mask]
                target = target[mask]

                y_predicted += logits.tolist()
                y_true += target.tolist()
        
        acc = metrics.Evaluation(y_predicted, y_true)
        print("Evaluation: " + str(acc))
        """
        Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label 
        You being i-th class and predicted label being j-th class.
        """
        try:
            test_labels = ['high_marsh', 'low_marsh']
            conf_mat = metrics.confusion_matrix(y_true, y_predicted, test_labels)
            print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
        except:
            labels = ['Background', 'high_marsh', 'low_marsh']
            conf_mat = metrics.confusion_matrix(y_true, y_predicted, labels)
            print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
            

        end_time = time.time()
        time_diff = end_time - start_time
        print("time spent: {}".format(time_diff))


In [23]:
def gen_weights(class_distribution, c = 1.02):
    return 1/torch.log(c + class_distribution)

class_distr = torch.Tensor([0.8345623386613673, 0.16543766133863266])
weight = gen_weights(class_distr, c=1.03)

In [24]:
weight

tensor([1.6051, 5.6019])